# Introduction to Machine Learning and Deep Learning

### Acknowledgements

The content of this notebook was originally created by Nils Eckstein, Julia Buhmann, and Jan Funke for the 2021 DL@MBL course in Woods Hole, and later chopped up and modified by Florian Jug for the 2021 course DL4MIA.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, Dense

### Let's get the MNIST data...

This is one of the most famous and most frequently used datasets of small images of hand-written digits and their corresponding ground-truth classes.

In this exercise we will learn to predict the correct class given an image of a hand-written digit.

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

"""
Returns:
2 tuples:

x_train, x_test: uint8 array of grayscale image data with shape (num_samples, 28, 28).
y_train, y_test: uint8 array of digit labels (integers in range 0-9) with shape (num_samples,).
"""

# Show example data
plt.subplot(1,4,1)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,2)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,3)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,4)
plt.imshow(x_train[3], cmap=plt.get_cmap('gray'))
plt.show()

### Let's create a network we'd like to train...

The one currently implemented in the cell below will turn out to not work so well. Run it anyways, but then come back here and start playing with changing the network architecture and hopefully find a better working model for the task at hand!

In [ ]:
input_shape = (x_train.shape[1], x_train.shape[2], 1)

cnn_model = Sequential()
cnn_model.add(Conv2D(filters=1,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=input_shape))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(filters=2,
                     kernel_size=(3, 3),
                     activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(10, activation='softmax')) # softmax for classification

cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad', # adaptive optimizer (still similar to SGD)
                  metrics=['accuracy'])

cnn_model.summary()

### Brind data in the shape we need during training...

In particular, this cell performs the following:
 * add a channel dimension to train and test data
 * normalize the pixel intensities to [0,1]
 * transform the ground-truth label from a digit (0, ..., 9) to one-hot encoded vectors. (Example: the one-hot encoded vector for digit `3` will become `0001000000`, etc.)

In [ ]:
from tensorflow.keras.utils import to_categorical

# add a channel dimension to the images
x_train = x_train.reshape(x_train.shape[0],
                          x_train.shape[1],
                          x_train.shape[2],
                          1)
x_test = x_test.reshape(x_test.shape[0],
                        x_test.shape[1],
                        x_test.shape[2],
                        1)

# rescale intensities to be between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert the labels into one-hot encodings
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

### Train the network...

Note that we decided on some things like the number of epochs, or a batch_size... what is all this? Could we have chosen other values? What would change?

In [ ]:
cnn_model.fit(x_train,
             y_train_onehot,
             steps_per_epoch=1000,
             batch_size=16,
             epochs=3,
             verbose=1,
             validation_data=(x_test, y_test_onehot)) # never actually validate using test data!

In [ ]:
score = cnn_model.evaluate(x_test, y_test_onehot, verbose=0)
print('MNIST test set accuracy:', score[1])

# visualize some test data and network output
y_predict = cnn_model.predict(x_test, verbose=0)
y_predict_digits = [np.argmax(y_predict[i]) for i in range(y_predict.shape[0])]


plt.subplot(1,4,1)
plt.imshow(x_test[0,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[0])
plt.subplot(1,4,2)
plt.imshow(x_test[1,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[1])
plt.subplot(1,4,3)
plt.imshow(x_test[2,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[2])
plt.subplot(1,4,4)
plt.imshow(x_test[3,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[3])

print("CNN predictions: {0}, {1}, {2}, {3}".format(y_predict_digits[0],
                                                   y_predict_digits[1],
                                                   y_predict_digits[2],
                                                   y_predict_digits[3]))

### Show a few more examples, so we also see some of the errors...

In [ ]:
fig, axes = plt.subplots(5, 8, figsize=(20,15))
errors=0
for i in range(40):
    if y_predict_digits[i]==y_test[i]:
        axes.flatten()[i].imshow(x_test[i,:,:,0], cmap=plt.get_cmap('gray'))
        axes.flatten()[i].set_title('ok')
    else:
        axes.flatten()[i].imshow(x_test[i,:,:,0], cmap=plt.get_cmap('Reds'))
        axes.flatten()[i].set_title('PREDICTION: %s'%(y_predict_digits[i]))
        errors+=1
        
print("Errors: %d"%errors)

### NEXT: improve the network, maybe also the network training, and reduce the test errors...

Another fun exercise might be to try to find the smallest network (fewest trainable parameters) that still leads to an test error smaller 15%...

**Note:** You will find out that if you modify the network, TensorFlow might hick up... likely you want to restart the kernel every time you make changes to the network...

### Once you're done, please anser these question:

 * What did you play with, what made the biggest difference?
 * How many parameters did you end up unsing?
 * How long did you train the network?
 * What was the best test-error you got overall?